In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
no_of_patients=109

In [3]:
train=[]
test=[]
for i in range(1,no_of_patients+1):
    files=glob('C:/aditya/Research_Intership/eeg_files/files/S'+str(str(i).zfill(3))+'/*.edf')
    files=files[2:3]
    train+=files
train

['C:/aditya/Research_Intership/eeg_files/files/S001\\S001R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S002\\S002R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S003\\S003R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S004\\S004R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S005\\S005R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S006\\S006R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S007\\S007R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S008\\S008R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S009\\S009R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S010\\S010R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S011\\S011R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S012\\S012R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S013\\S013R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S014\\S014R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S015\\S015R03.e

In [4]:
def read_data(i,train_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
#     display(eeg_df)
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample()
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
        
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    
    return eeg_df1,eeg_df2, len(eeg_df1),len(eeg_df2)

In [5]:
def scale_dataset(dataframe):
    x=dataframe[dataframe.columns[:-1]].values
    y=dataframe[dataframe.columns[-1]].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 80-20

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
ytemp1=[]
ytemp2=[]
for i in range(no_of_patients):
    xtr,xte,ytr,yte=read_data(train[i],0.8) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    ytemp1.append(ytr)
    ytemp2.append(yte)

xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)

ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        

xtrain['id']=ytrain
xtest['id']=ytest
display(xtrain)
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)


In [17]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dropout(0.2),
        Dense(128, activation = 'relu', name = "L3"),
        Dropout(0.2),
        Dense(64, activation = 'relu',   name = "L4"),
        Dropout(0.2),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'softmax', name = "L6"),
    ]
)

In [8]:
print(y)

[  0   0   0 ... 108 108 108]


In [18]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x,y,epochs=10
)

Epoch 1/10
40875/40875 [==============================] - 120s 3ms/step - loss: 1.9698
Epoch 2/10
40875/40875 [==============================] - 117s 3ms/step - loss: 1.4222
Epoch 3/10
40875/40875 [==============================] - 118s 3ms/step - loss: 1.2595
Epoch 4/10
40875/40875 [==============================] - 119s 3ms/step - loss: 1.1687
Epoch 5/10
40875/40875 [==============================] - 121s 3ms/step - loss: 1.1057
Epoch 6/10
40875/40875 [==============================] - 127s 3ms/step - loss: 1.0598
Epoch 7/10
40875/40875 [==============================] - 126s 3ms/step - loss: 1.0221
Epoch 8/10
40875/40875 [==============================] - 178s 4ms/step - loss: 0.9902
Epoch 9/10
40875/40875 [==============================] - 165s 4ms/step - loss: 0.9608
Epoch 10/10
40875/40875 [==============================] - 157s 4ms/step - loss: 0.9351


In [19]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L1 (Dense)                  (32, 64)                  4160      
                                                                 
 L2 (Dense)                  (32, 256)                 16640     
                                                                 
 dropout (Dropout)           (32, 256)                 0         
                                                                 
 L3 (Dense)                  (32, 128)                 32896     
                                                                 
 dropout_1 (Dropout)         (32, 128)                 0         
                                                                 
 L4 (Dense)                  (32, 64)                  8256      
                                                                 
 dropout_2 (Dropout)         (32, 64)                 

In [20]:
y_pred=model.predict(xt)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(yt,y_pred_classes))

10216/10216 [==============================] - 20s 2ms/step
              precision    recall  f1-score   support

           0       0.70      0.61      0.65      2999
           1       0.58      0.80      0.67      2999
           2       0.51      0.55      0.53      2999
           3       0.96      0.92      0.94      2999
           4       0.96      0.98      0.97      2999
           5       0.85      0.83      0.84      2999
           6       0.95      0.89      0.92      2999
           7       0.81      0.78      0.79      2999
           8       0.73      0.71      0.72      2999
           9       0.47      0.80      0.59      2999
          10       0.72      0.80      0.76      2999
          11       0.89      0.72      0.79      2999
          12       0.65      0.59      0.62      2999
          13       0.57      0.60      0.59      2999
          14       0.48      0.44      0.46      2999
          15       0.90      0.31      0.46      2999
          16       0.

In [31]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [32]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x,y,epochs=20
)

Epoch 1/20
40875/40875 [==============================] - 119s 3ms/step - loss: 1.1961
Epoch 2/20
40875/40875 [==============================] - 120s 3ms/step - loss: 0.5216
Epoch 3/20
40875/40875 [==============================] - 118s 3ms/step - loss: 0.3732
Epoch 4/20
40875/40875 [==============================] - 118s 3ms/step - loss: 0.2981
Epoch 5/20
40875/40875 [==============================] - 120s 3ms/step - loss: 0.2544
Epoch 6/20
40875/40875 [==============================] - 128s 3ms/step - loss: 0.2237
Epoch 7/20
40875/40875 [==============================] - 124s 3ms/step - loss: 0.2029
Epoch 8/20
40875/40875 [==============================] - 127s 3ms/step - loss: 0.1862
Epoch 9/20
40875/40875 [==============================] - 131s 3ms/step - loss: 0.1728
Epoch 10/20
40875/40875 [==============================] - 137s 3ms/step - loss: 0.1627
Epoch 11/20
40875/40875 [==============================] - 139s 3ms/step - loss: 0.1538
Epoch 12/20
40875/40875 [================

In [33]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

10216/10216 [==============================] - 79s 8ms/step
              precision    recall  f1-score   support

           0       0.88      0.83      0.85      2999
           1       0.88      0.88      0.88      2999
           2       0.75      0.62      0.68      2999
           3       0.84      0.97      0.90      2999
           4       0.97      0.99      0.98      2999
           5       0.99      0.95      0.97      2999
           6       0.95      0.95      0.95      2999
           7       0.93      0.74      0.82      2999
           8       0.41      0.88      0.56      2999
           9       0.67      0.81      0.73      2999
          10       0.92      0.77      0.84      2999
          11       0.91      0.91      0.91      2999
          12       0.67      0.92      0.78      2999
          13       0.83      0.87      0.85      2999
          14       0.70      0.89      0.78      2999
          15       0.85      0.63      0.72      2999
          16       0.